# Master AI Gateway Lab - Cleanup

This notebook deletes all resources deployed by the master lab.

## ⚠️ WARNING
Running this notebook will **permanently delete** all master lab resources:
- 3x AI Foundry Hubs + Projects
- 1x API Management instance
- 1x Redis Cache
- 1x Content Safety service
- 1x Azure Search service
- 1x Cosmos DB account
- 7x Container Apps
- 1x Container Registry
- 1x Log Analytics + App Insights

**This action cannot be undone!**

### Initialize Variables

In [1]:
import os, sys
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = 'master-lab-deployment'
resource_group_name = 'lab-master-lab'  # change if you used a different name

utils.print_ok('Cleanup initialized')

✅ Cleanup initialized ⌚ 12:14:05.097868 


### Verify Azure CLI Connection

In [2]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 12:14:13.890126 :8s]
👉🏽 Current user: lproux@microsoft.com
👉🏽 Tenant ID: 2b9d9f47-1fb6-400a-a438-39fe7d768649
👉🏽 Subscription ID: d334f2cd-3efd-494e-9fd3-2470b1a13e4c


### List Resources in Resource Group
Review what will be deleted before proceeding

In [3]:
import json

output = utils.run(f"az resource list --resource-group {resource_group_name}", 
                  f"Listed resources in {resource_group_name}", 
                  f"Failed to list resources")

if output.success and output.json_data:
    print(f"\nFound {len(output.json_data)} resources in resource group '{resource_group_name}':\n")
    for resource in output.json_data:
        print(f"  - {resource['type']}: {resource['name']}")
    print(f"\n⚠️  All {len(output.json_data)} resources will be deleted!")
else:
    utils.print_warning(f"Resource group '{resource_group_name}' may not exist or is empty")

⚙️ Running: az resource list --resource-group lab-master-lab 
✅ Listed resources in lab-master-lab ⌚ 12:14:29.604766 :15s]
⚠️ Resource group 'lab-master-lab' may not exist or is empty ⌚ 12:14:29.605403 


### ⚠️ FINAL CONFIRMATION

Set `CONFIRM_DELETE = True` in the cell below to proceed with deletion.

In [6]:
# Set this to True to confirm deletion
CONFIRM_DELETE = True

if not CONFIRM_DELETE:
    utils.print_warning("⚠️  Deletion not confirmed. Set CONFIRM_DELETE = True to proceed.")
    raise Exception("Deletion not confirmed. Stopping execution.")
else:
    utils.print_warning("✅ Deletion confirmed. Proceeding...")

⚠️ ✅ Deletion confirmed. Proceeding... ⌚ 12:15:26.788027 


### Delete Resource Group

This will delete the entire resource group and all contained resources.

**Estimated time:** 10-15 minutes

In [7]:
import time

utils.print_warning(f"Starting deletion of resource group '{resource_group_name}'...")
utils.print_info("This may take 10-15 minutes. Please wait...")

output = utils.run(f"az group delete --name {resource_group_name} --yes --no-wait",
                  f"Deletion initiated for resource group '{resource_group_name}'",
                  f"Failed to delete resource group '{resource_group_name}'")

if output.success:
    utils.print_ok("Deletion command sent successfully")
    utils.print_info("Monitor deletion progress in Azure Portal or run:")
    print(f"    az group show --name {resource_group_name}")
    utils.print_info("\nDeletion is running in the background.")
else:
    utils.print_error("Failed to initiate deletion")

⚠️ Starting deletion of resource group 'lab-master-lab'... ⌚ 12:25:33.554295 
👉🏽 This may take 10-15 minutes. Please wait...
⚙️ Running: az group delete --name lab-master-lab --yes --no-wait 
✅ Deletion initiated for resource group 'lab-master-lab' ⌚ 12:25:38.227381 :4s]
✅ Deletion command sent successfully ⌚ 12:25:38.227807 
👉🏽 Monitor deletion progress in Azure Portal or run:
    az group show --name lab-master-lab
👉🏽 
Deletion is running in the background.


### (Optional) Monitor Deletion Progress

Run this cell to check if the resource group has been fully deleted.

In [8]:
output = utils.run(f"az group show --name {resource_group_name}",
                  "",
                  "")

if output.success:
    utils.print_warning(f"Resource group '{resource_group_name}' still exists. Deletion in progress...")
else:
    utils.print_ok(f"✅ Resource group '{resource_group_name}' has been successfully deleted!")

⚙️ Running: az group show --name lab-master-lab 
✅ ✅ Resource group 'lab-master-lab' has been successfully deleted! ⌚ 12:25:42.596518 


## Cleanup Complete!

All master lab resources have been deleted. You can now:
1. Redeploy the master lab using `master-deployment.bicep`
2. Deploy individual labs from the parent `labs/` directory
3. Close this notebook

Thank you for using the Master AI Gateway Lab! 🎉